# Function Calling/Tool Use


## Just a quick definition

Function calling in the context of Large Language Models (LLMs) refers to the ability of the model to invoke external functions or tools during its response generation. Instead of only generating text, the LLM can recognize when a task requires external data or computation, call a predefined function with the appropriate arguments, and then use the function's output to continue its response.

Think of having a super smart robot that you can only speak with. Pretty useless at doing anything aside from talking. Now imagine you have given that robot a hammer and some nails. It can now put up that wall painting that's been waiting forever to be hung :D

Except, the way we as AI Engineers give AI tools is by defining python functions and describing that function in detail to the LLM so it knows what tools it has access to, what each tool can do, what are its inputs and expected outputs.

Just remember, a *tool* in its simplest form is just a *python function* that you have defined. It can be as simple as a calculator function or something like being able to call external APIs.

![tool-use](../images/tool-use.png)

*Image courtesy of [API Deck](https://www.apideck.com/blog/llm-tool-use-and-function-calling)*

## Step 1: Import libraries

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if OPENAI_API_KEY is None:
    raise Exception("API key is missing")

## Step 2: Define a tool

In [ ]:
import requests

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

In [ ]:
get_weather(24.343627, 54.497922)

## Step 3: Call a chat model normally

In [ ]:
client = OpenAI()

response = client.responses.create(
    model="gpt-4.1-nano",
    input=[
        {"role": "user", "content": "What is the weather like in Abu Dhabi now?"}
    ]
)

print(response.output_text)

In [ ]:
response.__dict__

In [ ]:
response.output[0].__dict__

## Step 4: Define the input schema for our tool

Lets try giving our `get_weather` function as a tool to our AI.

We will first need to define the tool in a format OpenAI can understand

In [ ]:
tools = [{
    'type': 'function',
    'name': 'get_weather',
    'description': 'Fetch the current weather for a specific location',
    'parameters': {
        'type': 'object',
        'properties': {
            'latitude': {'type': 'number'},
            'longitude': {'type': 'number'}
        },
        'required': ['latitude', 'longitude'],
        'additionalProperties': False
    },
    'strict': True
}]

## Step 5: Pass the tool schema over to the model

In [ ]:
input_messages = [{"role": "user", "content": "What is the weather like in Abu Dhabi now?"}]

In [ ]:
response = client.responses.create(
    model="gpt-4.1-nano",
    input=input_messages,
    tools=tools
)

In [ ]:
print(response.output_text)

In [ ]:
response.__dict__

In [ ]:
response.output[0].__dict__

## Step 6: Format the tool call response from the LLM

In [ ]:
import json

tool_call = response.output[0]
args = json.loads(tool_call.arguments)

In [ ]:
print(tool_call)

In [ ]:
print(args)

## Step 7: Pass on the tool call arguments to our tool/python function

We now need to pass on the arguments received by the model to our python function or tool

In [ ]:
result = get_weather(args['latitude'], args['longitude'])
result

## Step 8: Append the response of the tool into the message list

In [ ]:
input_messages.append(tool_call)

input_messages.append({
    "type": "function_call_output",
    "call_id": tool_call.call_id,
    "output": str(result)
})

In [ ]:
from pprint import pprint
pprint(input_messages)

## Step 9: Pass the message list into the model

In [ ]:
response_2 = client.responses.create(
    model="gpt-4.1-nano",
    input=input_messages,
    tools=tools
) 

In [ ]:
print(response_2.output_text)

In [ ]:
response_2.__dict__

# Resources:

- [OpenAIs Function Calling Guide](https://platform.openai.com/docs/guides/function-calling?api-mode=responses)
- [Anthropics Tool Use Guide with Claude](https://docs.anthropic.com/en/docs/agents-and-tools/tool-use/overview)
- [Function Calling with Gemini API](https://ai.google.dev/gemini-api/docs/function-calling?example=meeting)

<div style="border-radius:16px;background:#1e2a1e;margin:1em 0;padding:1em 1em 1em 3em;color:#eceff4;position:relative;box-shadow:0 6px 16px rgba(0,0,0,.4)">
  <b style="color:#a3be8c;font-size:1.25em">Your Challenge:</b>
  <div style="position:absolute;top:-.8em;left:-.8em;width:2.4em;height:2.4em;border-radius:50%;background:#a3be8c;color:#2e3440;display:flex;align-items:center;justify-content:center;font-weight:700;font-size:1.2em">💪</div>

  <br>
  Now that you've seen how to define a Python function as a tool and connect it to an LLM, it's time to get creative!

  <b>Pick your challenge</b><br>
  <b>1. Wikipedia Article Summarizer</b><br>
  - Fetch a wikipedia article you are interested in learning about and have an LLM summarize it<br>
  - <a href="https://pypi.org/project/Wikipedia-API/">Wikipedia API</a><br>

  <b>2. News Summarizer</b><br>
  - Fetch latest headlines or news by topic.<br>
  - <a href="https://newsapi.org">NewsAPI</a><br>
  - <a href="https://currentsapi.services/en/docs/">CurrentsAPI</a><br>

  <b>3. Stock Market Data</b><br>
  - Get real-time stock prices or company info<br>
  - <a href="https://www.alphavantage.co/">Alpha Vantage</a><br>
  - <a href="https://pypi.org/project/yfinance/">yfinance</a><br>

  <b>4. Movie Info</b><br>
  - Get movie ratings, cast, plot summaries.<br>
  - <a href="https://imdbapi.dev/">IMDb</a><br>
  - <a href="https://www.omdbapi.com/">OMDb API</a><br>

  <b>5. NASA API</b><br>
  - Astronomy facts, country data, etc.<br>
  - <a href="https://api.nasa.gov/">NASA APIs</a><br>

  <b>6. Your own custom tool</b><br>
  - Think of a real-world use case where an LLM could benefit from calling a custom function.<br>

  <hr>

  <b>Tips</b>:<br>
  - Use clear function names and docstrings.<br>
  - Handle input arguments and outputs carefully.<br>
  - Print the LLM's tool call and your function's output in a readable way.<br>

  Be sure to place your submissions in <code>part1-fundamentals/community-contributions/&lt;your-name&gt;</code><br>

  I'm super excited to see what you come up with :D
</div>